In [24]:
from platform import python_version
print(python_version())

3.6.8


In [25]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install sklearn

# Restricted Boltzman Machine 

A restricted Boltzmann Machine is an "Energy Based" generative stochastic model. They were initially invented by Paul Smolensky in 1986 and were called "Harmonium". After the evolution of training algorithms in the mid 2000's by Geoffrey Hinton, the boltzman machine became more prominent. They gained big popularity in recent years in the context of the Netflix Prize where RBMs achieved state of the art performance in collaborative filtering and have beaten most of the competition.

RBM's are useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling. 


## Architecture
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. As we move forward we would learn why the output layer won't be needed.
<img src="figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [30]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state

In [31]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [32]:
# graded
# import data

df = pd.read_excel('amazon.xlsx')

In [33]:
#run this and check if you have got the correct output
df.head()

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, you find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. You train your RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, let's convert the train and test data you split above into a bag of words model.

In [34]:
#graded
# create bag of words model on train and test data

tf=CountVectorizer(stop_words='english') #the final shape should be (number of documents, vocabulary)

# fit tf on the dataframe df
tf = tf.fit(df['Text'])

# transform df dataframe
trainX = tf.transform(df['Text'])

In [35]:
df['Text'].apply(lambda x: x.lower())
vectorizer = CountVectorizer()
trainX = vectorizer.fit_transform(df.iloc[:,0])
print(trainX)


  (0, 1491)	1
  (0, 1609)	1
  (0, 854)	1
  (0, 1074)	1
  (0, 1766)	1
  (0, 653)	1
  (0, 993)	1
  (0, 1640)	1
  (0, 1212)	1
  (0, 857)	1
  (0, 814)	2
  (0, 762)	1
  (0, 1604)	1
  (0, 1714)	1
  (0, 1702)	1
  (0, 711)	1
  (0, 233)	1
  (0, 367)	1
  (1, 715)	1
  (1, 259)	1
  (1, 569)	1
  (1, 1733)	1
  (2, 653)	1
  (2, 1604)	1
  (2, 721)	1
  :	:
  (997, 474)	1
  (997, 858)	1
  (997, 989)	1
  (998, 854)	1
  (998, 993)	1
  (998, 1604)	2
  (998, 1602)	1
  (998, 1109)	1
  (998, 1614)	1
  (998, 1323)	1
  (998, 453)	1
  (998, 829)	1
  (998, 1221)	1
  (998, 852)	1
  (999, 1604)	1
  (999, 1843)	1
  (999, 1081)	1
  (999, 1812)	1
  (999, 243)	1
  (999, 1065)	1
  (999, 240)	1
  (999, 1824)	1
  (999, 1105)	1
  (999, 1699)	1
  (999, 95)	1


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [37]:
#check if you are getting the correct output
print(sum(trainX.toarray()[1]))
trainX.shape

4


(1000, 1847)

You should get:
<br>

3 <br>
(1000, 1825)

Now that you have the bag of words model, let's define the number of visible and hidden units.

In [38]:
#graded
# define visible units
visibleUnits = trainX.shape[1] # vocabulary size ~1 line

# assign number of units
hiddenUnits = 5 # hyperparameter, this means that we are looking for 5 topics

In [39]:
visibleUnits

1847

In [40]:
#graded
# utility Functions
import math
# deine the sigmoid function
def sigmoid(x):
    #print(x)
    return 1 / (1 + np.exp(-x))

## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

The joint distribution is known as the Boltzmann Distribution which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and conditional probabilities of state v given the state h:
<img src="figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

Let's look at *sampleHiddenLayer* now.

### Sample Hidden Layer

You already know that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

In [41]:
#graded
def sampleHiddenLayer(v_sample):
    # write the code for calculation of hPdf using vectorized implementation of Eq 4
    
    hPdf = hiddenBias.reshape(hiddenBias.shape[0],1) + np.dot(W.transpose(),v_sample.transpose())
    #print("hPdf###: ", hPdf)
    signoid_func = np.vectorize(sigmoid)
    #print("signoid_func: ", signoid_func)
    hPdf = signoid_func(hPdf)
    #print("hPdf***: ", hPdf)
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, you are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
    

In [48]:
#graded
def sampleVisibleLayer(h_sample, D):
    # complete the following function such that vPdf has the sum of entries equal to 1 for each of the datapoints in the batch
    # you have to use axis = 1 in writing the denominator
    X = visibleBias.reshape(visibleBias.shape[0],1) + np.dot(W,h_sample)
    #print(X)
    numerator = np.exp(X)# ~1 line
    #print("numerator : ",numerator)
    denominator = numerator.sum(axis=1).reshape(numerator.shape[0],1)# ~1 line
    #print("denominator : ",denominator)
    vPdf = numerator/denominator# ~1 line
    #print("vPdf : ",vPdf)
    
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((vPdf.shape[0],batchSize))
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
    v_sample = v_sample.transpose()
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. Note that we are calculating the visible layer samples first and then using it to calculate he hidden layer sample. It'll become clear soon why we are doing so when you write the function for Contrastive Divergence.
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [49]:
#graded
def gibbs(h_sample, D):
    
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample,D)
    hPdf, h_sample = sampleHiddenLayer(v_sample)
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

You have learned that Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [50]:
#graded
def cd_k(data,k):
    
    D = data.sum(axis=1)
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data)# sample the hidden layer using the input data
    chain_start = hiddenSample_data

    for step in range(k):
        #print('step',step)
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start, D)# perform gibbs sampling using chain_start
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples, D)# perform gibbs sampling using hiddenSamples
    return hiddenPDF_data, visibleSamples, hiddenPDF


## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. **Figure out** the vectorized implementation for this.

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>
These will be helpful in writing the weight updates.

In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

In [51]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 100
eta = 0.05 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)
visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

# we shall use 10 interations of gibbs sampling
k=10

### Training

In [52]:
#graded
def train(dataX,k):
    
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs # calling the variables initiliazed at the begining
    
    mW = W # initialise momentum_weights
    mvisibleBias =  visibleBias # initialise momentum_visiblebiases
    mhiddenBias =  hiddenBias # initialise momentum_hiddenbiases
    for epoch in range(epochs):
        #print('epoch ',epoch)
        np_rng.shuffle(dataX) #shuffling the data
        
        for i in range(0, dataX.shape[0], batchSize):
            mData = dataX[i:i+batchSize,:]#select a batch of datapoints
            hiddenPDF_data, visibleSamples, hiddenPDF =  cd_k(mData,k) #perfrom Contrastive Divergence on the batch for k iterations
            
            W_diff = np.dot(mData.transpose(),hiddenPDF_data.transpose()) - np.dot(visibleSamples.transpose(),hiddenPDF.transpose())
            print("W_diff : ", W_diff)
            visible_bias_diff =  np.average(mData - visibleSamples,axis=0)
            hidden_bias_diff = np.average(hiddenPDF_data - hiddenPDF,axis=1)
            mW = mrate*mW - W_diff #write the momentum update equation for weight matrix
            mvisibleBias = mrate*mvisibleBias - visible_bias_diff #write the momentum update equation for visiblebias vector
            mhiddenBias =  mrate*mhiddenBias - hidden_bias_diff #write the momentum update equation for hiddenbias vector

            W = W + eta*mW    #weight update equation
            print("W : ", W)
            visibleBias = visibleBias + eta*mvisibleBias#visible bias update equation
            hiddenBias =  hiddenBias + eta*mhiddenBias#hidden bias update equation

## Train the Model

This will take around 10 minutes of time to run.

In [53]:
train(trainX.toarray(),k)

W_diff :  [[-3.49057843 -3.50056465 -3.59207664 -3.45020108 -3.47921335]
 [-7.0316884  -6.98740073 -7.07429984 -6.87615655 -6.92873068]
 [-8.60813544 -8.50407401 -8.64534338 -8.35286115 -8.4765571 ]
 ...
 [ 2.96590775  3.01108172  3.03245124  3.0003879   3.06124134]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
W :  [[ 0.16546826  0.18143028  0.19519291  0.16730867  0.17930297]
 [ 0.34918607  0.35446916  0.35614424  0.3342809   0.33130074]
 [ 0.44575773  0.42908627  0.43396931  0.4146382   0.41613894]
 ...
 [-0.14322792 -0.14423559 -0.15398124 -0.14579377 -0.14464717]
 [-0.00303036  0.00437621  0.00150838  0.00762721 -0.00342506]
 [ 0.00892952 -0.01333849  0.01601571  0.00696958 -0.01663715]]
W_diff :  [[-4.38896770e+00 -4.37439803e+00 -4.38845671e+00 -4.33872947e+00
  -4.37649278e+00]
 [-1.00431486e+01 -1.00333245e+01 -1.00441350e+01 -9.96155479e+00
  -1.00040816e+01]
 [-1.88249125e+00 -1.87512345e+00 -1.

W_diff :  [[ -1.00298718  -1.00306141  -1.00307691  -1.00317823  -1.00311998]
 [-13.         -13.         -13.         -13.         -13.        ]
 [-14.         -14.         -14.         -14.         -14.        ]
 ...
 [  0.82436096   0.81416936   0.81585772   0.82746159   0.82234868]
 [  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.        ]]
W :  [[ 7.35802638e+00  7.37451455e+00  7.39434422e+00  7.35537813e+00
   7.37288426e+00]
 [ 1.00783208e+01  1.00805649e+01  1.00876375e+01  1.00471456e+01
   1.00510148e+01]
 [ 1.05695202e+01  1.05479165e+01  1.05594394e+01  1.05249471e+01
   1.05333782e+01]
 ...
 [-7.91459888e-01 -7.92667657e-01 -7.98638059e-01 -8.00022025e-01
  -7.98144589e-01]
 [-3.33530496e-03  4.25953604e-03  1.32378881e-03  7.57129153e-03
  -3.73708190e-03]
 [ 3.49449564e-03 -1.91198504e-02  1.06871494e-02  1.52166532e-03
  -2.26455873e-02]]
W_diff :  [[-11.0002978  -11.00029094 -11.00027879 -11.

W_diff :  [[-2.         -2.         -2.         -2.         -2.        ]
 [-4.         -4.         -4.         -4.         -4.        ]
 [-3.         -3.         -3.         -3.         -3.        ]
 ...
 [ 1.04107634  1.04076459  1.04142303  1.04117591  1.04201269]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
W :  [[ 1.66296978e+01  1.66461986e+01  1.66660348e+01  1.66270828e+01
   1.66445791e+01]
 [ 2.00086461e+01  2.00108842e+01  2.00179674e+01  1.99774387e+01
   1.99813216e+01]
 [ 1.99063005e+01  1.98858139e+01  1.98971083e+01  1.98641056e+01
   1.98727131e+01]
 ...
 [-1.58893928e+00 -1.58783097e+00 -1.59367137e+00 -1.59524027e+00
  -1.59302868e+00]
 [-3.33590095e-03  4.25930776e-03  1.32342791e-03  7.57118190e-03
  -3.73769172e-03]
 [ 3.33624480e-03 -1.92758670e-02  1.05248399e-02  1.36249643e-03
  -2.28056412e-02]]
W_diff :  [[-1.00000000e+01 -1.00000000e+01 -1.00000000e+01 -1.00000000e+01
  -1.0000

W_diff :  [[-1.50000000e+01 -1.50000000e+01 -1.50000000e+01 -1.50000000e+01
  -1.50000000e+01]
 [-3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
  -3.00000000e+00]
 [-1.80000000e+01 -1.80000000e+01 -1.80000000e+01 -1.80000000e+01
  -1.80000000e+01]
 ...
 [ 1.02658326e+00  1.02694985e+00  1.02674772e+00  1.02647287e+00
   1.02650099e+00]
 [ 7.51488107e-28  7.69864996e-28  7.66685718e-28  8.16678178e-28
   7.94045839e-28]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
W :  [[ 2.43474154e+01  2.43639163e+01  2.43837525e+01  2.43448005e+01
   2.43622968e+01]
 [ 2.70600725e+01  2.70623106e+01  2.70693939e+01  2.70288651e+01
   2.70327480e+01]
 [ 2.68272161e+01  2.68069321e+01  2.68181804e+01  2.67854563e+01
   2.67941384e+01]
 ...
 [-2.37355653e+00 -2.37198996e+00 -2.37758840e+00 -2.37959677e+00
  -2.37744578e+00]
 [-3.33590212e-03  4.25930732e-03  1.32342720e-03  7.57118169e-03
  -3.73769291e-03]
 [ 3.33564974e-03 -1.92764561e-02  1.05

W_diff :  [[-4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
  -4.00000000e+00]
 [-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-1.10000000e+01 -1.10000000e+01 -1.10000000e+01 -1.10000000e+01
  -1.10000000e+01]
 ...
 [ 5.72018130e-10  5.72712734e-10  5.46446087e-10  5.82255775e-10
   6.25939183e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
W :  [[ 2.98187391e+01  2.98352399e+01  2.98550762e+01  2.98161242e+01
   2.98336205e+01]
 [ 3.37619307e+01  3.37641688e+01  3.37712521e+01  3.37307233e+01
   3.37346062e+01]
 [ 3.43892208e+01  3.43689806e+01  3.43802108e+01  3.43475447e+01
   3.43562641e+01]
 ...
 [-3.04217384e+00 -3.04047833e+00 -3.04603620e+00 -3.04842248e+00
  -3.04607244e+00]
 [-3.33590212e-03  4.25930732e-03  1.32342720e-03  7.57118169e-03
  -3.73769291e-03]
 [ 3.33564813e-03 -1.92764577e-02  1.05

...

C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


W_diff :  [[-2.70000000e+01 -2.70000000e+01 -2.70000000e+01 -2.70000000e+01
  -2.70000000e+01]
 [-4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
  -4.00000000e+00]
 [-5.00000000e+00 -5.00000000e+00 -5.00000000e+00 -5.00000000e+00
  -5.00000000e+00]
 ...
 [ 4.01950657e-31  4.01978966e-31  3.89349064e-31  4.01656053e-31
   4.23616321e-31]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.91151972e-37  2.87580919e-37  3.00498250e-37  2.99694783e-37
   2.93389475e-37]]
W :  [[ 1.06710802e+02  1.06727302e+02  1.06747139e+02  1.06708187e+02
   1.06725683e+02]
 [ 9.93931085e+01  9.93953466e+01  9.94024299e+01  9.93619011e+01
   9.93657840e+01]
 [ 1.15833840e+02  1.15813833e+02  1.15825047e+02  1.15792646e+02
   1.15801777e+02]
 ...
 [-9.14002077e+00 -9.13703069e+00 -9.14121937e+00 -9.14576237e+00
  -9.14267339e+00]
 [-3.33590212e-03  4.25930732e-03  1.32342720e-03  7.57118169e-03
  -3.73769291e-03]
 [ 3.33564813e-03 -1.92764577e-02  1.05

W_diff :  [[-1.90000000e+01 -1.90000000e+01 -1.90000000e+01 -1.90000000e+01
  -1.90000000e+01]
 [-3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
  -3.00000000e+00]
 [-4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
  -4.00000000e+00]
 ...
 [ 7.75600524e-06  7.87457795e-06  7.83390316e-06  7.71466876e-06
   7.73583860e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
W :  [[ 1.16609414e+02  1.16625915e+02  1.16645751e+02  1.16606799e+02
   1.16624295e+02]
 [ 1.08342250e+02  1.08344489e+02  1.08351572e+02  1.08311043e+02
   1.08314926e+02]
 [ 1.24110984e+02  1.24090978e+02  1.24102192e+02  1.24069791e+02
   1.24078924e+02]
 ...
 [-9.78746926e+00 -9.78447839e+00 -9.78866686e+00 -9.79320955e+00
  -9.79011915e+00]
 [-3.33590212e-03  4.25930732e-03  1.32342720e-03  7.57118169e-03
  -3.73769291e-03]
 [ 3.33564813e-03 -1.92764577e-02  1.05

C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
C:\Users\home\Anaconda3\envs\python3.6.8\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater_equal


ValueError: pvals < 0, pvals > 1 or pvals contains NaNs

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. You can take a look at the words under each topic and see what they are talking about. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. You can see that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary
    
Feel free to change the number of iterations of gibbs sampling in Contrastive Divergence and see how the distribution of words change under the topics.

In [ ]:
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias)
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten()
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            lm+=1
            if lm==15:
                break
        print('\n')

In [ ]:
worddist( hiddenUnits, vectorizer.vocabulary_)

The output shows the probability assigned to each word for ever topic present.